# Functionality: generate simulated 2-d locations of all channels

 <ul>
  <li>M1: 96 channels</li>
  <li> SMATha: 27 channels 
      (base on the channel information of ThaSMA in 
       <a href = "../../../data/pinky_chnInfThaSMA.csv">data\pinky_chnInfThaSMA.csv</a>
      )
      <ul>
          <li>left SMA (lSMA):         4 channels </li>
          <li>right SMA (rSMA):        4 channels</li>
          <li>left motor cortex(lMC):  2 channels </li>
          <li>right Caudate(rCd):      2 channels </li>
          <li>left VA (lVA):           3 channels </li>
          <li>right VA (rVA):          2 channels </li>
          <li>left VLo (lVLo)          2 channels </li>
          <li>right VLo (rVLo)         2 channels </li>
          <li>left VPLo (lVPLo):       3 channels</li>
          <li>right VPLo (rVPLo):      3 channels</li>        
      </ul>
  </li>
</ul> 

## Packages import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from decimal import Decimal

import pickle
import os, sys

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# add path '../'
sys.path.append(os.path.abspath(os.path.join('..')))

# import animal_folder in ../util/folder_extract.pynb 
from util.folder_extract import animal_folder

importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Codes/FCAnalysis/util/folder_extract.ipynb


## Generate Coordinates

### circle points generation function 

In [2]:
def circle_points(r, n, center = [0,0]):
    """
        generate evenly distrubted circle points
        
        Args:
            
            r: list of radius of each circle (e.g r = [0.02, 0.04, 0.06, 0.08])
            
            n: list of number on each circle (e.g n = [3, 17, 31, 45])
    """
    
    circles = np.empty(shape=[0,2])
    for r, n in zip(r, n):
        # evenly spaced t over a [0 2*pi]
        t = np.linspace(0, 2*pi, n + 1)
        
        # x, y coordinates
        x = r * np.cos(t) + center[0]
        y = r * np.sin(t) + center[1]
        coord = np.stack((x[:-1], y[:-1]), axis = -1)
        
        circles = np.append(circles, coord, axis = 0)
    
    
    ids = np.uint16(np.linspace(1, np.sum(n), num=np.sum(n)))
        
    return circles, ids

 ### clusterpoints (<4) generation function

In [3]:
def cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02):
    """
        generate cluster points for points less than four
        
        Args:
            xy_lefttop: (x, y) coordinates of the left top point
            
            npoints: the number of points
            
            space_x, space_y: spaces between two points in x and y axis
            
    """
    
    # the x, y coordinates of the first point
    x_init, y_init = xy_lefttop[0], xy_lefttop[1]

    # number of points each line
    nx = 2

    # extract x, y coordinates of each point
    coord_points = np.empty(shape=[0,2])
    for i in range(npoints):

        # steps from the first point in x and y axis
        step_x, step_y = i%nx, i//nx

        # x, y coordinates
        x, y = x_init + step_x * space_x, y_init - step_y * space_y

        coord_points = np.append(coord_points, np.expand_dims([x, y], axis = 0), axis = 0)

    
    if npoints == 3:
        "if there are 3 points, adjust the last to be a triangular"
        
        coord_points[2][0] = (coord_points[0][0] + coord_points[1][0])/2
    
    # solve the float problem: -0.05+0.02 = -0.030000000000000002
    points = np.round(coord_points * 100) /100
    
    ids = np.uint16(np.linspace(start = 1, stop = npoints, num = npoints))

    return coord_points, ids

### generate coordinats 

1. call function circle_points()

2. generate coordinates of area M1, ThaSMA, STN and GP

In [4]:
def genCoord_ThaSMA(x0 = -0.15, y0 = -0.15):
    """
        generate coordinates of all the SMA, MC, VA, VLo, VLPo and Cd 
        
        Args:
        
        Return:
            
            coord_ThaSMA, x, y coordinates of all channels in SMA, MC, VA, VLo, VLPo and Cd
            <class 'numpy.ndarray'>, shape = (nchns, 2)
            
            labels_ThaSMA: labels of ThaSMA for each pair of coordinates <list>, len = nchns
    """
    
    # using ThaSMA to contain all the SMA, MC, VA, VLo, VLPo and Cd 
    coord_ThaSMA, labels_ThaSMA = np.empty((0,2)), list()

    x_space, y_space = 0.1, 0.05


    """
        first layer: SMA
    """

    ## lSMA ##
    # parameters setup for lSMA
    x_lSMA, y_lSMA = x0, y0
    npoints, xy_lefttop = 4, (x_lSMA, y_lSMA)

    # generate coord_points of lSMA
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for lSMA
    labels = ["lSMA" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels



    ## rSMA ##
    # parameters setup for rSMA
    x_rSMA, y_rSMA = x_lSMA + x_space, y_lSMA
    npoints, xy_lefttop = 4, (x_rSMA, y_rSMA)

    # generate coord_points of rSMA
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for rSMA
    labels = ["rSMA" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels


    """
        second layer: lMC
    """

    ## lMC ##
    # parameters setup for lMC
    x_lMC, y_lMC = x0, np.min(coord_ThaSMA[:,1]) - y_space
    npoints, xy_lefttop = 2, (x_lMC, y_lMC)

    # generate coord_points of lMC
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for lMC
    labels = ["lMC" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels


    """
        third layer: VA
    """

    ## lVA ##
    # parameters setup for lVA
    x_lVA, y_lVA = x0, np.min(coord_ThaSMA[:,1]) - y_space
    npoints, xy_lefttop = 3, (x_lVA, y_lVA)

    # generate coord_points of lVA
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for lVA
    labels = ["lVA" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels


    ## rVA ##
    # parameters setup for rVA
    x_rVA, y_rVA = x_lVA + x_space, y_lVA
    npoints, xy_lefttop = 2, (x_rVA, y_rVA)

    # generate coord_points of rVA
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for rVA
    labels = ["rVA" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels




    """
        fourth layer: VLo
    """

    ## lVLo ##
    # parameters setup for lVLo
    x_lVLo, y_lVLo = x0, np.min(coord_ThaSMA[:,1]) - y_space
    npoints, xy_lefttop = 2, (x_lVLo, y_lVLo)

    # generate coord_points of lVLo
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for lVLo
    labels = ["lVLo" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels



    ## rVLo ##
    # parameters setup for rVLo
    x_rVLo, y_rVLo = x_lVLo + x_space, y_lVLo
    npoints, xy_lefttop = 2, (x_rVLo, y_rVLo)

    # generate coord_points of rVLo
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for rVLo
    labels = ["rVLo" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels



    """
        fifth layer: VPLo
    """

    ## lVPLo ##
    # parameters setup for lVPLo
    x_lVPLo, y_lVPLo = x0, np.min(coord_ThaSMA[:,1]) - y_space
    npoints, xy_lefttop = 3, (x_lVPLo, y_lVPLo)

    # generate coord_points of lVPLo
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for lVPLo
    labels = ["lVPLo" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels



    ## rVPLo ##
    # parameters setup for rVPLo
    x_rVPLo, y_rVPLo = x_lVPLo + x_space, y_lVPLo
    npoints, xy_lefttop = 3, (x_rVPLo, y_rVPLo)

    # generate coord_points of rVPLo
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for rVPLo
    labels = ["rVPLo" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels




    """
        sixth layer: rCd
    """

    ## rCd ##
    # parameters setup for rCd
    x_rCd, y_rCd = x0 + x_space, np.min(coord_ThaSMA[:,1]) - y_space
    npoints, xy_lefttop = 2, (x_rCd, y_rCd)

    # generate coord_points of rCd
    coord_points, ids = cluster_points(npoints, xy_lefttop, space_x = 0.02, space_y = 0.02)

    # labels for rCd
    labels = ["rCd" + str(idi) for idi in ids]

    # concatenate into coord_ThaSMA, and labels_ThaSMA
    coord_ThaSMA = np.concatenate((coord_ThaSMA, coord_points), axis = 0)
    labels_ThaSMA = labels_ThaSMA + labels

    # del temporal variables
    del npoints, xy_lefttop, coord_points, ids, labels
    
    
    
    
    return coord_ThaSMA, labels_ThaSMA

In [5]:
def genCoord():
    
    """
        generate coordinates of area M1, ThaSMA, STN and GP
        
        Return:
            coord_simulated: generated simulated coordinates of 
            M1 (96 channels), 
            
            lSMA(4 channels), rSMA(4 channels), lMC(2 channels), rCd(2 channels), lVA(3 channels), 
            rVA(2 channels), lVLo(2 channels),  rVLo(2 channels), lVPLo(3 channels), rVPLo(3 channels), 
            
            STN (7 channels) and GP (7channels) 
            
            dict_keys(['coord_M1', 'labels_M1', 'coord_ThaSMA', 'labels_ThaSMA', 
            'coord_STN', 'labels_STN', 'coord_GP', 'labels_GP'])
    """

    # coordinates of M1 (96 channels)
    r = [0.02, 0.04, 0.06, 0.08]
    n = [3, 17, 31, 45]
    coord_M1, ids= circle_points(r, n, center = [0, 0])
    labels_M1 = ["M1-" + str(idi) for idi in ids]
    del r, n, ids

    # coordinates of STN
    coord_STN, ids = circle_points([0.03], [7], center = [0.2, -0.1])
    labels_STN = ["STN" + str(idi) for idi in ids]
    del ids

    # coordinates of GP
    coord_GP, ids = circle_points([0.03], [7], center = [0.15, -0.2])
    labels_GP = ["GP" + str(idi) for idi in ids]
    del ids
    
    
    ### coordinates of ThaSMA ###
    coord_ThaSMA, labels_ThaSMA = genCoord_ThaSMA(x0 = -0.15, y0 = -0.15)
    

    # store ciCOH_normal and ciCOH_mild into dict res_ciCOH
    coord_simulated = {'coord_M1': coord_M1, "labels_M1": labels_M1,
                       "coord_STN": coord_STN, "labels_STN": labels_STN,
                       "coord_GP": coord_GP, "labels_GP": labels_GP,
                       "coord_ThaSMA": coord_ThaSMA, "labels_ThaSMA": labels_ThaSMA}
    
    return coord_simulated

## Save  Simulated Coordinates

1. save simulated coordinates of M1, ThaSMA, STN and GP to animetafolder/coord_simulated.pickle

In [6]:
def saveSimCoord(coord_simulated):
    """
        save simulated coordinates of M1, ThaSMA, STN and GP  coord_simulated 
        to animetafolder/coord_simulated.pickle
    """

    # extract the metainf folder of animal
    _, _, animetafolder = animal_folder(animal = 'Pinky')

    # file storing the simulated coordinates
    coordfile = os.path.join(animetafolder, 'coord_simulated.pickle')

    pickle.dump(coord_simulated, open(coordfile, "wb" ) )

## Main

In [7]:
# generate coordinates of area M1, ThaSMA, STN and GP
coord_simulated  = genCoord()

# save the generated coord_simulated
saveSimCoord(coord_simulated)


## Plot Coordinates

In [ ]:
# plot coordinates of SMA
coord_ThaSMA, labels_ThaSMA  = genCoord_ThaSMA(x0 = -0.15, y0 = -0.15)
fig, ax = plt.subplots()
ax.scatter(coord_ThaSMA[0:8, 0], coord_ThaSMA[0:8,1], c='r')
ax.scatter(coord_ThaSMA[8:10, 0], coord_ThaSMA[8:10,1], c='k')
ax.scatter(coord_ThaSMA[10:15, 0], coord_ThaSMA[10:15,1], c='g')
ax.scatter(coord_ThaSMA[15:19, 0], coord_ThaSMA[15:19,1], c='b')
ax.scatter(coord_ThaSMA[19:25, 0], coord_ThaSMA[19:25,1], c='c')
ax.scatter(coord_ThaSMA[25:, 0], coord_ThaSMA[25:,1], c='y')

In [ ]:
# plot coordinates of all channels
coord_simulated = genCoord()

fig, ax = plt.subplots()

for key in coord_simulated:
    
    if "coord" in key:
        ax.scatter(coord_simulated[key][:,0], coord_simulated[key][:,1])
        
ax.set_aspect('equal')